<a href="https://colab.research.google.com/github/sudarshan-koirala/youtube-stuffs/blob/main/mistralai_with_and_without_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MistralAI API Usage
- [Console link](https://console.mistral.ai/)

- [Mistral Pricing](https://docs.mistral.ai/platform/pricing)
- [Openai Pricing](https://openai.com/pricing)

In [ ]:
%%capture
!pip install mistralai llama-index watermark

In [49]:
%reload_ext watermark
%watermark -a "Sudarshan Koirala" -vmp llama_index,mistralai

Author: Sudarshan Koirala

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

llama_index: 0.9.15.post2
mistralai  : 0.0.8

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.58+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [ ]:
# get API key -> https://console.mistral.ai/users/api-keys/
import os
os.environ['MISTRAL_API_KEY'] ="MISTRAL_API_KEY"
api_key = os.environ["MISTRAL_API_KEY"]

[MistralAI Platform](https://docs.mistral.ai/platform/overview)
- Client Code
- Endpoints
- Guardrailing
- Pricing

## Chat Completions

In [50]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

model = "mistral-tiny"

client = MistralClient(api_key=api_key)

messages = [
    ChatMessage(role="user", content="What is the capital of Nepal?")
]

# No streaming
chat_response = client.chat(
    model=model,
    messages=messages,
)

chat_response

ChatCompletionResponse(id='cmpl-44b1a630f3df4239b5ce851f92fe864a', object='chat.completion', created=1702826188, model='mistral-tiny', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='The capital city of Nepal is Kathmandu. It is the largest city and the political and cultural center of the country. Located in the central part of Nepal, Kathmandu is famous for its rich history, beautiful temples, and vibrant markets. Its metropolitan area has a population of about 1.5 million people.'), finish_reason=<FinishReason.stop: 'stop'>)], usage=UsageInfo(prompt_tokens=16, total_tokens=87, completion_tokens=71))

In [51]:
# With streaming
for chunk in client.chat_stream(model=model, messages=messages):
    print(chunk)

id='cmpl-989ee6a40854409d9e6d9a16fc582563' model='mistral-tiny' choices=[ChatCompletionResponseStreamChoice(index=0, delta=DeltaMessage(role='assistant', content=None), finish_reason=None)] created=None object=None usage=None
id='cmpl-989ee6a40854409d9e6d9a16fc582563' model='mistral-tiny' choices=[ChatCompletionResponseStreamChoice(index=0, delta=DeltaMessage(role=None, content='The capital city of Nep'), finish_reason=None)] created=1702826216 object='chat.completion.chunk' usage=None
id='cmpl-989ee6a40854409d9e6d9a16fc582563' model='mistral-tiny' choices=[ChatCompletionResponseStreamChoice(index=0, delta=DeltaMessage(role=None, content='al is Kathmandu. It is the largest city and'), finish_reason=None)] created=1702826216 object='chat.completion.chunk' usage=None
id='cmpl-989ee6a40854409d9e6d9a16fc582563' model='mistral-tiny' choices=[ChatCompletionResponseStreamChoice(index=0, delta=DeltaMessage(role=None, content=' the political and cultural center of the country. Kathmandu is'), f

## Embeddings

In [52]:
from mistralai.client import MistralClient

client = MistralClient(api_key=api_key)

embeddings_batch_response = client.embeddings(
      model="mistral-embed",
      input=["The capital city of Nepal is Kathmandu."],
  )

In [53]:
# get the length of embeddings
first_embedding = embeddings_batch_response.data[0].embedding
length_of_embedding = len(first_embedding)
print(length_of_embedding)

1024


In [54]:
# first 5 dimensions
first_embedding[:5]

[-0.0099029541015625,
 0.0233001708984375,
 0.03973388671875,
 0.01375579833984375,
 0.038421630859375]

# [Guardrailing](https://docs.mistral.ai/platform/guardrailing)
- Toggling `safe_mode` will prepend your messages with the following system prompt
```
Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.
```

In [55]:
client = MistralClient(api_key=api_key)

messages = [
    ChatMessage(role="user", content="What is the capital of Nepal?")
]

chat_response = client.chat(
    model="mistral-tiny",
    messages= messages,
    safe_mode=True
)

chat_response

ChatCompletionResponse(id='cmpl-5a6579cd008f4bd1b3c2cbdea070f1a2', object='chat.completion', created=1702826332, model='mistral-tiny', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='The capital city of Nepal is Kathmandu. It is the largest city and the political and cultural center of the country. Kathmandu is located in the central part of Nepal and is known for its rich history, beautiful temples, and vibrant markets.'), finish_reason=<FinishReason.stop: 'stop'>)], usage=UsageInfo(prompt_tokens=62, total_tokens=117, completion_tokens=55))

# MistralAI with LlamaIndex

## MistralAI Embedding Model

In [56]:
from llama_index.embeddings import MistralAIEmbedding

In [57]:
model_name = "mistral-embed"
embed_model = MistralAIEmbedding(model_name=model_name, api_key=api_key)

embeddings = embed_model.get_text_embedding("The capital of Nepal is Kathmandu.")

In [58]:
embeddings[:5]

[-0.0015850067138671875,
 0.022247314453125,
 0.04425048828125,
 0.0074310302734375,
 0.036407470703125]

## MistralAI LLMs

#### Call `complete` with a prompt

In [59]:
from llama_index.llms import MistralAI

llm = MistralAI(api_key=api_key)
resp = llm.complete("Paul Graham is ")

In [60]:
resp

CompletionResponse(text='Paul Graham is a well-known entrepreneur, hacker, and essayist. He co-founded the startup incubator Y Combinator in 2005, which has since become one of the most prominent startups accelerators in the world. Graham is also known for his essays on entrepreneurship, programming, and startups, which have been published on his website, Hacker News, and in various publications. He has been described as a "pioneer of the startup scene in Silicon Valley" and a "leading figure in the Y Combinator startup community." Graham\'s essays have influenced many entrepreneurs and startups, and he is considered a thought leader in the tech industry.', additional_kwargs={}, raw={'id': 'cmpl-b850e1fb78274e88a39b960c8f5fc859', 'object': 'chat.completion', 'created': 1702826448, 'model': 'mistral-tiny', 'choices': [ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='Paul Graham is a well-known entrepreneur, hacker, and essayist. He co-founded the star

#### Call `chat` with a list of messages

In [61]:
from llama_index.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are CEO of MistralAI."),
    ChatMessage(role="user", content="Tell me the story about La plateforme"),
]
resp = llm.chat(messages)

In [62]:
print(resp)

assistant: Once upon a time, in the heart of Paris, France, a team of passionate researchers and engineers came together with a bold vision: to build an artificial intelligence (AI) company that would revolutionize the way businesses and organizations create, innovate, and grow. This team formed the core of Mistral AI.

La plateforme, as we came to call it, was the flagship project of Mistral AI. It was an ambitious endeavor to build a cutting-edge AI platform that would enable businesses to harness the power of AI and deep learning technologies, without requiring extensive expertise in these fields. Our goal was to make AI accessible to everyone, regardless of their size or industry.

The journey to create La plateforme was filled with challenges and breakthroughs. Our team spent countless hours researching, experimenting, and refining our approach. We drew inspiration from the latest advancements in AI and deep learning, as well as the unique needs and challenges of businesses.

One 

## Streaming

Using `stream_complete` endpoint

In [63]:
resp = llm.stream_complete("Paul Graham is ")

In [64]:
for r in resp:
    print(r.delta, end="")

Paul Graham is a well-known entrepreneur, hacker, and essayist. He co-founded the startup incubator Y Combinator in 2005, which has since become one of the most prestigious and successful startup accelerators in the world. Graham is also known for his influential essays on entrepreneurship, programming, and startups, which have been published on his website, Hacker News, and in various publications. He has been described as a "pioneer of the startup scene in Silicon Valley" and a "leading figure in the Y Combinator startup community." Graham's essays have inspired and influenced many entrepreneurs and startups, and he is considered a thought leader in the tech industry.

In [65]:
from llama_index.llms.types import ChatMessage

messages = [
    ChatMessage(role="system", content="You are CEO of MistralAI."),
    ChatMessage(role="user", content="Tell me the story about La plateforme"),
]
resp = llm.stream_chat(messages)

In [66]:
for r in resp:
    print(r.delta, end="")

As the CEO of MistralAI, I am proud to share the story of our flagship product, La plateforme. La plateforme, which translates to "The Platform" in English, is a cutting-edge artificial intelligence (AI) solution designed to revolutionize the way businesses operate and interact with their customers.

The idea for La plateforme was born out of a simple yet profound observation: the world was becoming increasingly digital, and businesses needed a powerful, adaptable, and intelligent tool to navigate this new reality. Our team of visionary engineers, data scientists, and business strategists came together with a shared goal: to create an AI platform that could understand and learn from human interactions, provide personalized recommendations, and automate routine tasks.

The development of La plateforme was a long and arduous journey. Our team spent countless hours researching the latest AI technologies, experimenting with different architectures, and refining our algorithms. We drew insp

## Configure Model

In [67]:
llm = MistralAI(model="mistral-medium", api_key=api_key)

In [68]:
resp = llm.stream_complete("Paul Graham is ")

In [69]:
for r in resp:
    print(r.delta, end="")

Paul Graham is a well-known figure in the tech industry. He is a computer programmer, venture capitalist, and essayist. Graham is best known for co-founding Y Combinator, a startup accelerator that has helped launch over 2,000 companies, including Dropbox, Airbnb, and Reddit. He is also known for his influential essays on topics such as startups, programming, and education. Before starting Y Combinator, Graham co-founded Viaweb, an online store builder that was acquired by Yahoo in 1998. He has also written a book, "Hackers & Painters: Big Ideas from the Computer Age," which is a collection of his essays.

## Async

In [70]:
resp = await llm.acomplete("Paul Graham is ")

In [71]:
print(resp)

Paul Graham is a well-known figure in the technology and startup world. He is a computer scientist, entrepreneur, venture capitalist, and essayist. Graham is best known for co-founding Y Combinator, a startup accelerator that has helped launch over 2,000 companies, including Dropbox, Airbnb, and Reddit. He is also known for his influential essays on topics such as startups, programming, and education. Before co-founding Y Combinator, Graham co-founded Viaweb, an online store builder that was acquired by Yahoo in 1998. He has also written a book, "Hackers & Painters: Big Ideas from the Computer Age."
